In [1]:
%%time
import spacy
from spacy.tokens import Doc
from spacy.vocab import Vocab

CPU times: user 573 ms, sys: 72.8 ms, total: 645 ms
Wall time: 645 ms


In [2]:
def preclean_text(text):
    text = text.replace('\n', '')
    text = '_ _ ' + text.replace('.', '. _ _ ')
    text = text.replace('  ', ' ')
    
    return text

In [3]:
%%time
textcorpus = ''

for i in range(1, 203):
    textcorpus += (open('CORPUS/{}.txt'.format(i)).read().split('Contenido:')[-1])
    
textcorpus = preclean_text(textcorpus)

textcorpus[:1000]

CPU times: user 20.8 ms, sys: 13.4 ms, total: 34.2 ms
Wall time: 34 ms


'_ _ La moral es un sistema de control para la integración grupal, un decálogo explicativo impuesto para asemejar a los “buenos” y excluir a los “malos”. _ _ Suele ser muy mala con los individuos que quieren ser diferentes y genuinos, la más mínima desobediencia despierta su ira. _ _ Su definición del “bien” y del “mal” es ad hoc, dependiendo de los rasgos culturales del grupo y de la religión que lo funda. _ _ La ética es una liberación de la moral para interrogar libre y sensatamente qué ha de ser universalmente celebrado en la existencia. _ _ La ética no explica nada, interroga desde la sed de universalidad, y crea personas autónomas, capaces de crear nuevas maneras de coexistir, sin imposición. _ _ Desde luego, la ética es un acontecimiento escaso, improbable y frágil. _ _ Es para los desobedientes, como Jesús, Gandhi, Mandela. _ _ La moral detesta la ética. _ _ La ética mira a la moral con tristeza, le da pena su dogmatismo y su violencia familiar. _ _ Del mismo modo, las religion

In [4]:
%%time
nlp = spacy.load('es_core_news_md')
nlp.max_length = 2000000

CPU times: user 16.8 s, sys: 626 ms, total: 17.4 s
Wall time: 17.5 s


In [5]:
%%time
doc = nlp(textcorpus)
doc.vocab.to_disk('corpus_vocab')
doc.to_disk('corpus_doc')
del doc

CPU times: user 51.9 s, sys: 11.4 s, total: 1min 3s
Wall time: 55.5 s


In [6]:
%%time
vocab = Vocab().from_disk('corpus_vocab')
doc = Doc(vocab).from_disk('corpus_doc')

CPU times: user 12.5 s, sys: 703 ms, total: 13.2 s
Wall time: 13.2 s


In [7]:
offset = doc[0]

In [8]:
[sent for sent in doc.sents][0]

_ _ La moral es un sistema de control para la integración grupal, un decálogo explicativo impuesto para asemejar a los “buenos” y excluir a los “malos”.

In [9]:
def get_trigrams(sentences):
    # Lista para almacenar los trigramas
    trigrams = []
    tag_trigrams = []
    
    # Recorrer la lista de palabras
    for sent in sentences:
        for i, _ in enumerate(sent):
            try:
                w1, w2, w3 = sent[i:i+3]
            except:
                try:
                    w1, w2 = sent[i:i+2]
                    w3 = offset
                except:
                    w1 = sent[i]
                    w2 = w3 = offset
            
            # Agregar trigrama a la lista
            trigrams.append((w1.text, w2.text, w3.text))
            
            tag_trigrams.append((
                w1.tag_,
                w2.tag_,
                w3.tag_
            ))
    
    return trigrams, tag_trigrams

In [10]:
def get_word_trigrams(sentences):
    # Lista para almacenar los trigramas
    trigrams = []
    
    # Recorrer la lista de palabras
    for sent in sentences:
        for i, _ in enumerate(sent):
            try:
                w1, w2, w3 = sent[i:i+3]
            except:
                try:
                    w1, w2 = sent[i:i+2]
                    w3 = offset
                except:
                    w1 = sent[i]
                    w2 = w3 = offset
            
            # Agregar trigrama a la lista
            trigrams.append((w1.text, w2.text, w3.text))
    
    return trigrams

In [11]:
%%time
word_trigrams, tag_trigrams = get_trigrams(doc.sents)

CPU times: user 2.4 s, sys: 90.2 ms, total: 2.49 s
Wall time: 2.49 s


In [12]:
word_trigrams[:10]

[('_', '_', 'La'),
 ('_', 'La', 'moral'),
 ('La', 'moral', 'es'),
 ('moral', 'es', 'un'),
 ('es', 'un', 'sistema'),
 ('un', 'sistema', 'de'),
 ('sistema', 'de', 'control'),
 ('de', 'control', 'para'),
 ('control', 'para', 'la'),
 ('para', 'la', 'integración')]

In [13]:
tag_trigrams[:10]

[('PROPN___',
  'PROPN___',
  'DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art'),
 ('PROPN___',
  'DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art',
  'NOUN__Gender=Fem|Number=Sing'),
 ('DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art',
  'NOUN__Gender=Fem|Number=Sing',
  'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin'),
 ('NOUN__Gender=Fem|Number=Sing',
  'AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
  'DET__Definite=Ind|Gender=Masc|Number=Sing|PronType=Art'),
 ('AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
  'DET__Definite=Ind|Gender=Masc|Number=Sing|PronType=Art',
  'NOUN__Gender=Masc|Number=Sing'),
 ('DET__Definite=Ind|Gender=Masc|Number=Sing|PronType=Art',
  'NOUN__Gender=Masc|Number=Sing',
  'ADP__AdpType=Prep'),
 ('NOUN__Gender=Masc|Number=Sing',
  'ADP__AdpType=Prep',
  'NOUN__Gender=Masc|Number=Sing'),
 ('ADP__AdpType=Prep', 'NOUN__Gender=Masc|Number=Sing', 'ADP__AdpType=Prep'),
 ('NOUN__Gender=Masc|Number=Sing',
 

In [14]:
def build_model(trigrams):
    model = {}
    
    # Contamos la frecuencia de co-ocurrencia
    for i, _ in enumerate(trigrams):
        w1, w2, w3 = trigrams[i]
    
        # El control de excepciones se encarga de manejar los distintos casos 
        # en que un trigrama aún no ha sido registrado.
        try:
            model[w1, w2][w3] += 1
        except: # Aqui se asume que w3 lanza la excepcion
            try:
                model[w1, w2][w3] = 1
            except: # Aqui se asume que el par (w1, w2) lanza la excepcion
                model[w1, w2] = {w3:1}
            
    # Ahora transformamos el conteo en probabilidades
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
    
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
            
    return model

In [15]:
%%time
word_model = build_model(word_trigrams)

CPU times: user 483 ms, sys: 33.4 ms, total: 517 ms
Wall time: 517 ms


In [16]:
%%time
tag_model = build_model(tag_trigrams)

CPU times: user 286 ms, sys: 9.67 ms, total: 296 ms
Wall time: 296 ms


In [17]:
len(word_model)

126530

In [18]:
len(tag_model)

9064

In [19]:
%%time
word_min = 1

for i in word_model.values():
    for ii in i.values():
        if ii < word_min:
            word_min = ii
            
word_min

CPU times: user 39.1 ms, sys: 3.19 ms, total: 42.3 ms
Wall time: 42.1 ms


0.00016477179106936892

In [20]:
%%time
tag_min = 1

for i in tag_model.values():
    for ii in i.values():
        if ii < tag_min:
            tag_min = ii
            
tag_min

CPU times: user 6.81 ms, sys: 25 µs, total: 6.84 ms
Wall time: 6.71 ms


7.216048491845865e-05

In [21]:
def edits1(word):
    "Distancia de edición 1"
    letters = 'abcdefghijklmnñopqrstuvwxyzáéíóú'
    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    
    replaces = [L +  C + R[1:] for L, R in splits if R for C in letters]
    
    inserts = [L + C + R for L, R in splits for C in letters]
    
    return set(deletes + transposes + replaces + inserts)

In [22]:
def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [23]:
def get_dictionary():
    return set([w.text for w in doc])

In [24]:
%%time
dictionary = get_dictionary()

CPU times: user 329 ms, sys: 9.94 ms, total: 339 ms
Wall time: 339 ms


In [25]:
len(dictionary)

26465

In [26]:
def known(words):    
    return set(w for w in words if w in dictionary)

In [27]:
def suggestions(word):    
    word = word.lower()
    
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

### Probability

In [28]:
def P_Word(trigram):
    n_2, n_1, n = trigram
    
    try:
        return word_model[(n_2, n_1)][n]
    except:
        return word_min

In [29]:
def P_Tag(trigram):
    n_2, n_1, n = trigram
    
    try:
        return tag_model[(n_2, n_1)][n]
    except:
        return tag_min

In [30]:
def getsugg(trigrams):
    
    trigramlist = []
    
    for t in trigrams:
        w1, w2, w3 = t
        
        w1s = suggestions(w1)
        w2s = suggestions(w2)
        w3s = suggestions(w3)
        
        sugglist = []
        
        for i in w1s:
            for ii in w2s:
                for iii in w3s:
                    w_1, w_2, w_3 = nlp(i + ' ' + ii + ' ' + iii)
                    ptag = P_Tag((w_1.tag_, w_2.tag_, w_3.tag_))
                    pword = P_Word((i, ii, iii))
                    
                    sugglist.append({(i,ii,iii): (pword*ptag)})
                    
        trigramlist.append(sugglist)
                
    return trigramlist

In [31]:
def gettermax(tsugg):
    maxlist = []
    
    for s in tsugg:
        maxlist.append(max(s, key = lambda x : max(x.values()) ))
        
    return maxlist

In [32]:
def get_correction(text):
    
    text = preclean_text(text)
    
    doctest = nlp(text)
    trigramas_tests = get_word_trigrams(doctest.sents)
    
    ss = getsugg(trigramas_tests)
    
    common = gettermax(ss)
    
    w1, w2, w3 = [k for k in common[0].keys()][0]
    ttfinal = [w1, w2, w3]

    for i in range(1, len(common)):
        w1, w2, w3  = [k for k in common[i].keys()][0]
    
        ttfinal.append(w3)
    
        if [v for v in common[i].values()][0] > [v for v in common[i-1].values()][0]:
            ttfinal[i] = w1
            ttfinal[i+1] = w2
    
    result = ''
    last_w = ''
    
    for w in ttfinal:
        if last_w == '_':
            result += w.capitalize() + ' '
        else:
            result += w + ' '
            
        last_w = w
            
    result = result.replace('. _ _ ', '.')
    result = result.replace(' .', '. ')
    result = result.replace(' :', ': ')
    result = result.replace(' ,', ', ')
    result = result.replace(' ;', '; ')
    result = result.replace('¿ ', '¿')
    result = result.replace('¡ ', '¡')
    result = result.replace(' ?', '?')
    result = result.replace(' !', '!')
    result = result.replace('_ _ ', ' ')
    result = result.replace('  ', ' ')
    
    return result

In [33]:
%%time
test_text = """
Hay canbio de planes.
La probinsia orienttal.
El intrnet de las cosas.
La onra del artizta.
Para unrar a su amigo.
El camno del merkado.
La defenza de nuestros ideales.
Tod0 está en saver conbibir.
4lgunos alg0r1tmos pu3d3n s3r muy interes4nt3s.
"""

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.3 µs


In [34]:
%%time
aa = get_correction(test_text)

CPU times: user 4.06 s, sys: 0 ns, total: 4.06 s
Wall time: 4.09 s


In [35]:
aa

' Hay cambio de planes. La provincia oriental. El internet de las cosas. La obra del artista. Para curar a su amigo. El camino del mercado. La defensa de nuestros ideales. Todo está en saber convivir. Algunos algorítmos pueden ser muy interesantes. '